<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/geemap/tree/master/examples/template/template.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/geemap/blob/master/examples/template/template.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/geemap/blob/master/examples/template/template.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# Clasificación supervisada SVM
Script trabajado por Francisco Albornoz (2021-04-28 12:51:11 ) / Última modificación / Análisis y clasificación supervisada región de Aysen

<img src=https://www.ecopaysen.cl/wp-content/uploads/2019/12/ecopaysen_marca_web.png>

In [1]:
# Installs geemap package
# import subprocess

# try:
#     import geemap
# except ImportError:
#     print("Installing geemap ...")
#     subprocess.check_call(["python", "-m", "pip", "install", "geemap"])
# !pip install scikit_learn


In [2]:
import os
import ee
import geemap
import xarray_leaflet
import rasterio 
from rasterio import plot
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import pandas as pd

## Crear un mapa interactivo 
The default basemap is `Google Maps`. [Additional basemaps](https://github.com/giswqs/geemap/blob/master/geemap/basemaps.py) can be added using the `Map.add_basemap()` function. 

In [3]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

## Escogemos la imagen landsat 

In [4]:
centroid = ee.Geometry.Point([-72, -44])

landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(centroid) \
    .filterDate('2020-01-01', '2021-03-31') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')

landsat_vis = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(centroid, 8)
Map.addLayer(landsat, landsat_vis, "Landsat-8")


## Importar rasters locales 

In [5]:
%pwd

'/mnt/c/users/Usuario/Documents/GitHub/VitoAlbornoz/GEE'

In [6]:
# verificar en el sistema del pc cual es la ruta
# os.path.expanduser('/mnt/c/Users/R7 3700X/Documents/GitHub/VitoAlbornoz/') 
# os.path.expanduser('/mnt/c/Users/Usuario/Documents/Github/Vitoalbornoz/')

In [7]:
# verificar en el sistema del pc cual es la ruta
out_dir = os.path.join(os.path.expanduser('/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/'), 'Data')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

raster_aysen = os.path.join(out_dir, 'landcover_2018_aysen.tif')

In [8]:
out_dir

'/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/Data'

In [9]:
raster_aysen

'/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/Data/landcover_2018_aysen.tif'

In [10]:
Map.add_raster(raster_aysen, colormap= 'terrain', layer_name='Landcover 2018')

In [11]:
landcover = ee.Image("users/franciscoalbornoz/landcover_sur")
landcover

In [12]:
raster = rasterio.open(out_dir+'/landcover_2018_aysen.tif')

In [13]:
raster.bounds

BoundingBox(left=-73.169644655, bottom=-46.394042378, right=-70.719691266, top=-43.705745621)

In [14]:
raster.crs

CRS.from_epsg(4326)

In [15]:
raster

<open DatasetReader name='/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/Data/landcover_2018_aysen.tif' mode='r'>

In [16]:
mosaico = ee.Image("users/franciscoalbornoz/Mosaico_2018")
clasificacion_2018 = ee.Image("users/franciscoalbornoz/clasificacion_2018")

landsat_vis = {
    'min': 0,
    'max': 1,
    'bands': ['b5', 'b4', 'b3']
}
Map.addLayer(mosaico, landsat_vis, "Landsat-8-2018")
Map.addLayer(clasificacion_2018.randomVisualizer(), {}, "clasificación 2018")

Map

Map(center=[-44, -72], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

## Obteniendo los valores para el Training Datasets

In [17]:
work_dir = os.path.expanduser('../Data/shapes/')

merged_shp = os.path.join(work_dir, 'roi_h.shp')
val_shp= os.path.join(work_dir, 'validacion.shp')
merged_original =os.path.join(work_dir,'roi_merged.shp')

In [18]:
merged = geemap.shp_to_ee(merged_shp)
Map.addLayer(merged, {}, 'Áreas de entrenamiento')

In [19]:
val = geemap.shp_to_ee(val_shp)
Map.addLayer(val, {}, 'Áreas de Validación')

In [20]:
mergedoriginal =geemap.shp_to_ee(merged_original)
Map.addLayer(mergedoriginal, {}, 'Áreas de entrenamiento Original')

In [21]:
dataset = mosaico.sampleRegions(**{
    'collection': merged,
    'properties': ['IDH'],
    'scale' : 30 ,
})

In [22]:
dataset1 = mosaico.sampleRegions(**{
    'collection': mergedoriginal,
    'properties': ['ID'],
    'scale': 30,
})

In [23]:
datasetval = mosaico.sampleRegions(**{
    'collection': val,
    'properties': ['ID'],
    'scale': 10,
})

In [24]:
print(dataset1.size().getInfo())

14254


In [25]:
print(datasetval.size().getInfo())

3097


In [26]:
dataset.propertyNames()

In [27]:
datasetval.propertyNames()

In [28]:
print(dataset.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'IDH': 9, 'b1': 0.026168961077928543, 'b2': 0.04253531992435455, 'b3': 0.03470490872859955, 'b4': 0.33766427636146545, 'b5': 0.1495322287082672, 'b6': 0.06873124837875366, 'b7': 0.8135994672775269, 'b8': 0.7762474417686462, 'b9': 0.5209251642227173}}


In [29]:
print(datasetval.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'ID': 3, 'b1': 0.01979999989271164, 'b2': 0.017999999225139618, 'b3': 0.010499999858438969, 'b4': 0.01730000041425228, 'b5': 0.00989999994635582, 'b6': 0.0071000000461936, 'b7': 0.24460433423519135, 'b8': -0.019829994067549706, 'b9': 0.019325504079461098}}


In [30]:
# out_csv = os.path.join(out_dir, 'Dataset_entrenamiento.csv')
# geemap.extract_values_to_points(merged, mosaico, out_csv)

In [31]:
# out_csv2 =os.path.join(out_dir, 'Dataset_validacion.csv')
# geemap.extract_values_to_points(val, mosaico, out_csv2)

In [32]:
# df = pd.read_csv(out_csv)

In [33]:
# df

In [34]:
# df2 = pd.read_csv(out_csv2)

In [35]:
# df2

### Dividiendo en muestras y datos de validación

In [36]:
# Guardamos las bandas
bands = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']

# Esta propiedad guarda las clases de landcover
label = 'IDH'
label1 ='ID'
# Overlay the points on the imagery to get training.
# sample = landsat.select(bands).sampleRegions(**{
#   'collection': points,
#   'properties': [label],
#   'scale': 30
# })

# Adds a column of deterministic pseudorandom numbers. 
# sample = dataset.randomColumn()

# split = 0.7 

# training = sample.filter(ee.Filter.lt('random', split))
# validation = sample.filter(ee.Filter.gte('random', split))

In [37]:
# training.first().getInfo()

In [38]:
# validation.first().getInfo()

## Entrenando el modelo 

In [39]:
# from sklearn import svm
# from sklearn import ensemble

In [40]:
# feature_names = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']
# label = "IDH"

In [41]:
# X = df[feature_names]
# y= df[label]

In [42]:
classifi = ee.Classifier.libsvm('Voting','C_SVC','RBF', gamma = 0.5 , cost = 10 )

In [43]:
classifi1 = ee.Classifier.libsvm('Voting','C_SVC','RBF', gamma = 0.5 , cost = 10 )


In [44]:
trained = classifi.train(dataset, label, bands)
classified = mosaico.classify(trained)

In [45]:
trained1 = classifi.train(dataset1, label1, bands)
classified1 = mosaico.classify(trained1)

In [46]:
trained

In [47]:
classified

### Clasificando la imagen

In [48]:
class_values = [1,2,3,4,5,6,7,8,9,10,11]
class_values1 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
class_palette = ['0c40f1', #1 agua*
 'f0ff00', #2 cultivo*
 'f99504', #3 estepa*
 '17d4c6', #4 humedal*
 '24ff00', #5 matorral abierto*
 '2de50f', #6 matorral arborescente*
 'ccba7d', #7 plantaciones*
 'dec61a', #8 praderas*
 '28d909', #9 primario*
 '4e8c44', #10 secundario*
 '685205', #11 suelo desnudo*
]
class_palette1 = ['22c129',
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 '1c6330',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '28d909',
 '03c2ff',
 '4e8c44',
 'fdfdfd',
 'c0c233'
]

In [49]:
land = classified.set('classification_class_values', class_values)
landcoverf = land.set('classification_class_palette', class_palette)
Map.addLayer(landcoverf, {}, 'Land cover SVM')

In [50]:
land1 = classified1.set('classification_class_values', class_values1)
landcoverf1 = land1.set('classification_class_palette', class_palette1)
Map.addLayer(landcoverf1, {}, 'Land cover SVM original')


In [51]:
Map

Map(center=[-45.0498939995, -71.9446679605], controls=(WidgetControl(options=['position'], widget=HBox(childre…

### Categorizando según las clases 

In [52]:
# landcover.getInfo()

In [53]:
# class_values = [1,
#  2,
#  3,
#  4,
#  5,
#  6,
#  7,
#  8,
#  9,
#  10,
#  11,
#  12,
#  13,
#  14,
#  15,
#  16]

In [54]:
# class_palette = ['476ba1',
#  'd1defa',
#  'decaca',
#  'd99482',
#  'ee0000',
#  'ab0000',
#  'b3aea3',
#  '68ab63',
#  '1c6330',
#  'b5ca8f',
#  'a68c30',
#  'ccba7d',
#  'e3e3c2',
#  'caca78',
#  '99c247',
#  '78ae94']


In [55]:
# landcoverf = result.set('classification_class_values', class_values)
# landcoverf = landcoverf.set('classification_class_palette', class_palette)

In [56]:
# Map.addLayer(landcoverf, {}, 'Land cover clasificado' )
# Map

### Visualizando el resultado


In [57]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))


Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

In [58]:
Map

Map(center=[-45.0498939995, -71.9446679605], controls=(WidgetControl(options=['position'], widget=HBox(childre…

### Agregando título al mapa

In [59]:
Map.add_legend('Clasificación supervisada SVM')
Map

### Evaluación de presición / Accuracy assessment

In [60]:
train_accuracy = trained1.confusionMatrix()

In [61]:
train_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 600, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 4, 0, 0],
 [0, 1, 768, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 26],
 [0, 0, 0, 3019, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1428, 0, 7, 187, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 4, 76, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 909, 0, 0, 0, 0, 13, 3, 0, 6, 0, 0, 0, 0],
 [0, 0, 0, 0, 23, 0, 0, 598, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 49, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 84, 0, 0, 0, 0, 0, 0, 0, 0, 2],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 221, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 45, 0, 0, 0, 3, 108, 0, 0, 6, 0, 0, 0, 0],
 [0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 4, 3091, 0, 0, 0, 0, 0, 0],
 [0, 0, 28, 0, 6, 0, 0, 22, 0, 0, 0, 0, 0, 130, 0, 0, 0, 0, 72],
 [0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 24, 0, 744, 0, 2, 0, 0],
 [0, 0, 0, 122, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 95, 0, 0, 

In [62]:
train_accuracy.accuracy().getInfo()

0.9300547214816893

In [63]:
train_accuracy.kappa().getInfo()

0.91974796573592

In [64]:
train_accuracy.producersAccuracy().getInfo()

[[0],
 [0.96],
 [0.9528535980148883],
 [0.9996688741721854],
 [0.8798521256931608],
 [0],
 [0.9763694951664876],
 [0.9629629629629629],
 [0],
 [0.9767441860465116],
 [0.9954954954954955],
 [0.6666666666666666],
 [0.9977404777275661],
 [0.5038759689922481],
 [0.9526248399487837],
 [0.4377880184331797],
 [0.7397058823529412],
 [1],
 [0.9804878048780488]]

In [65]:
train_accuracy.consumersAccuracy().getInfo()

[[0,
  0.7722007722007722,
  0.9516728624535316,
  0.9463949843260188,
  0.9760765550239234,
  0,
  0.926605504587156,
  0.7355473554735548,
  0,
  1,
  0.9608695652173913,
  0.5217391304347826,
  0.9913406029506093,
  0.9774436090225563,
  0.9587628865979382,
  1,
  0.9673076923076923,
  0.9935897435897436,
  0.8893805309734514]]

#### Set de datos de validación

In [66]:
validated = datasetval.classify(trained1)

In [67]:
validated.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0_0',
 'properties': {'ID': 3,
  'b1': 0.01979999989271164,
  'b2': 0.017999999225139618,
  'b3': 0.010499999858438969,
  'b4': 0.01730000041425228,
  'b5': 0.00989999994635582,
  'b6': 0.0071000000461936,
  'b7': 0.24460433423519135,
  'b8': -0.019829994067549706,
  'b9': 0.019325504079461098,
  'classification': 3}}

In [68]:
test_accuracy = validated.errorMatrix('ID', 'classification')

In [69]:
test_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 36, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5],
 [0, 0, 0, 44, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0],
 [0, 0, 0, 0, 591, 0, 0, 27, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 4],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 23, 0, 0, 225, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 90],
 [0, 0, 0, 1, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 7, 21, 0, 8, 0, 0, 7, 0, 0, 20, 28, 21, 11, 10, 0, 1, 0, 8],
 [0, 59, 13, 0, 0, 0, 11, 0, 0, 0, 2, 85, 14, 0, 79, 0, 2, 0, 2],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 0, 7, 0, 0, 0, 0],
 [0, 0, 126, 0, 8, 0, 0, 0, 0, 0, 7, 22, 0, 50, 0, 0, 0, 0, 78],
 [0, 561, 25, 0, 8, 0, 5, 0, 0, 0, 0, 39, 12, 0, 412, 0, 38, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 

In [70]:
test_accuracy.accuracy().getInfo()

0.5095253471101066

In [71]:
test_accuracy.kappa().getInfo()

0.43841467330747014

In [72]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [1],
 [0.8780487804878049],
 [0.8627450980392157],
 [0.9395866454689984],
 [0],
 [0],
 [0.665680473372781],
 [0],
 [0.5],
 [0.14084507042253522],
 [0.31835205992509363],
 [0.78125],
 [0.1718213058419244],
 [0.37454545454545457],
 [0],
 [0.524390243902439],
 [0],
 [0]]

In [73]:
test_accuracy.consumersAccuracy().getInfo()

[[0,
  0.004491017964071856,
  0.16071428571428573,
  0.9777777777777777,
  0.8914027149321267,
  0,
  0,
  0.8687258687258688,
  0,
  1,
  0.6896551724137931,
  0.480225988700565,
  0.3246753246753247,
  0.7352941176470589,
  0.7587476979742173,
  0,
  0.6771653543307087,
  0,
  0]]

### Descargando la matrix

In [74]:
# import csv
# import os

# out_dir = os.path.join(os.path.expanduser('/mnt/c/Users/R7 3700X/'), 'Desktop')
# training_csv = os.path.join(out_dir, 'train_accuracy.csv')
# testing_csv = os.path.join(out_dir, 'test_accuracy.csv')

# with open(training_csv, "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerows(train_accuracy.getInfo())
    
# with open(testing_csv, "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerows(test_accuracy.getInfo())

### Exportando 

In [75]:
# import os
# out_dir = os.path.join(os.path.expanduser('/mnt/c/Users/R7 3700X/'), 'Desktop')
# out_file = os.path.join(out_dir, 'landcover.tif')

In [76]:
# geemap.ee_export_image(landcover, filename=out_file, scale=900)


In [77]:
# geemap.ee_export_image_to_drive(landcover, description='landcover', folder='export', scale=900)